# Using Styles

Styles are a collection of configurations that can be applied to a Deeplay module. Different modules will have different styles available.

In [1]:
import deeplay as dl
import torch

## Checking Available Styles

You can check the available styles using the `.available_styles()` method. This method can be called either on the class or on an instance of the class.

In [2]:
dl.Conv2dBlock.available_styles()

['residual',
 'spatial_self_attention',
 'spatial_cross_attention',
 'spatial_transformer',
 'resnet',
 'resnet18_input']

## Applying Styles

You can apply a style using the `.style()` method, which takes the name of the style as an argument. 

In [3]:
block = dl.Conv2dBlock(3, 64)

print(block)

Conv2dBlock(
  (layer): Layer[Conv2d](in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=0)
)


In [4]:
block = block.style("residual")

print(block)

Conv2dBlock(
  (shortcut_start): Conv2dBlock(
    (layer): Layer[Conv2d](in_channels=3, out_channels=64, kernel_size=1, stride=1, padding=0)
  )
  (blocks): Sequential(
    (0): Conv2dBlock(
      (layer): Layer[Conv2d](in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1)
      (activation): Layer[ReLU]()
      (normalization): Layer[BatchNorm2d](num_features=64)
    )
    (1): Conv2dBlock(
      (layer): Layer[Conv2d](in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
      (activation): Layer[ReLU]()
      (normalization): Layer[BatchNorm2d](num_features=64)
    )
  )
  (shortcut_end): Add()
)


It is fine to chain `.style()` with other configuration methods. However, it is recommended to apply the style first. Most styles assume that the module is in an unstyled state. Applying a style after other configurations may lead to unexpected results.

In [5]:
block = dl.Conv2dBlock(3, 64).style("residual").strided(2)

print(block)

Conv2dBlock(
  (shortcut_start): Conv2dBlock(
    (layer): Layer[Conv2d](in_channels=3, out_channels=64, kernel_size=1, stride=2, padding=0)
  )
  (blocks): Sequential(
    (0): Conv2dBlock(
      (layer): Layer[Conv2d](in_channels=3, out_channels=64, kernel_size=3, stride=2, padding=1)
      (activation): Layer[ReLU]()
      (normalization): Layer[BatchNorm2d](num_features=64)
    )
    (1): Conv2dBlock(
      (layer): Layer[Conv2d](in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
      (activation): Layer[ReLU]()
      (normalization): Layer[BatchNorm2d](num_features=64)
    )
  )
  (shortcut_end): Add()
)


Some styles may have additional parameters. These parameters can be passed as keyword arguments to the `.style()` method. Your editor should show you the available parameters as well as a documentation string for the style.

In [6]:
block = dl.Conv2dBlock(3, 64).style("residual", order="allaalla")

print(block)

Conv2dBlock(
  (shortcut_start): Conv2dBlock(
    (layer): Layer[Conv2d](in_channels=3, out_channels=64, kernel_size=1, stride=1, padding=0)
  )
  (blocks): Sequential(
    (0): Conv2dBlock(
      (activation): Layer[ReLU]()
      (layer): Layer[Conv2d](in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1)
    )
    (1): Conv2dBlock(
      (layer): Layer[Conv2d](in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
      (activation): Layer[ReLU]()
    )
    (2): Conv2dBlock(
      (activation): Layer[ReLU]()
      (layer): Layer[Conv2d](in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
    )
    (3): Conv2dBlock(
      (layer): Layer[Conv2d](in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
      (activation): Layer[ReLU]()
    )
  )
  (shortcut_end): Add()
)


## Extending Styles

You can easily create your own styles. This is useful to apply a set of configurations to multiple modules. To create a style, you simply need to use the `@class.register_style` decorator. The name of the function will be the name as the style. The function should take a module as the first argument, and any additional parameters as keyword arguments.

In [7]:
@dl.LinearBlock.register_style
def my_linear(block: dl.LinearBlock, bias_in_shortcut=False):
    block.activated(torch.nn.GELU)
    block.normalized(torch.nn.BatchNorm1d)
    block.shortcut(shortcut=dl.Layer(torch.nn.Linear, 
                                     block.in_features, 
                                     block.out_features, 
                                     bias=bias_in_shortcut))
    return block

dl.LinearBlock.available_styles()

['my_linear']

In [8]:
block = dl.LinearBlock(3, 64).style("my_linear")

print(block)

LinearBlock(
  (shortcut_start): Layer[Linear](in_features=3, out_features=64, bias=False)
  (layer): Layer[Linear](in_features=3, out_features=64, bias=True)
  (activation): Layer[GELU]()
  (normalization): Layer[BatchNorm1d]()
  (shortcut_end): Add()
)


In [9]:
block_with_bias = dl.LinearBlock(3, 64) \
    .style("my_linear", bias_in_shortcut=True)

print(block_with_bias)

LinearBlock(
  (shortcut_start): Layer[Linear](in_features=3, out_features=64, bias=True)
  (layer): Layer[Linear](in_features=3, out_features=64, bias=True)
  (activation): Layer[GELU]()
  (normalization): Layer[BatchNorm1d]()
  (shortcut_end): Add()
)
